In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/NN - Spring 2023/HW2"
!ls

/content/drive/MyDrive/Colab Notebooks/NN - Spring 2023/HW2
'Copy of main (3).py'   HW2_exps.ipynb		     oneLayer	     resnet18
 dataset	        images			     PetImages	     vgg16
 densenet121	        kagglecatsanddogs_5340.zip   randomForrest


In [7]:
import numpy as np
import pandas as pd
import os
import random
import time
import copy
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import random
from typing import List, Dict, Tuple
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import tqdm
import random
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from torchvision.models.feature_extraction import create_feature_extractor

# Utils

In [4]:
def mkdir(directory: str):
    if not os.path.exists(directory):
        os.makedirs(directory)

def plot_stats(train_loss_hist, test_loss_hist, 
               train_acc_hist, test_acc_hist,
               title: str,
               path: str=None, name_to_save: str=None, save_only=False):
    plt.clf()
    # plt.rcParams["figure.figsize"] = (16,7)
    plt.subplot(1, 2, 1)
    plt.plot(train_loss_hist, label = 'train loss')
    plt.plot(test_loss_hist, label = 'test loss')
    plt.title('loss')
    plt.legend()
    # plt.plot()
    # plt.show()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_acc_hist, label = 'train acc')
    plt.plot(test_acc_hist, label = 'test acc')
    plt.title('acc')
    plt.legend()
    
    plt.suptitle(f'{title}')
    if not path is None:
      mkdir(path)
      plt.savefig(f'{path}/{name_to_save}.png')
    
    if not save_only:
      plt.show()

In [ ]:
# !wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

In [ ]:
# !unzip -q kagglecatsanddogs_5340.zip

# Data loading

In [5]:
# https://stackoverflow.com/questions/54049440/speed-up-datasets-loading-on-google-colab
def get_data(batch_size: int = 8, randseed: int = 11, pin_memory=False,
             pret_model_data: str = 'ImageNet', num_workers=1):
    def seed_worker(worker_id):
        # worker_seed = torch.initial_seed() % 2 ** 32
        np.random.seed(randseed)
        random.seed(randseed)

    g = torch.Generator()
    g.manual_seed(randseed)

    def is_valid(filename: str):
        corrupted = [
            'PetImages/Cat/Thumbs.db',
            'PetImages/Dog/Thumbs.db',
            'PetImages/Cat/666.jpg',
            'PetImages/Dog/11702.jpg'
        ]
        return not (filename in corrupted)
    
    # from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
    if pret_model_data == 'ImageNet':
        img_dimensions = 224
        # https://discuss.pytorch.org/t/understanding-transform-normalize/21730
        # https://stackoverflow.com/a/75986472
        transform = transforms.Compose([
            transforms.Resize((img_dimensions, img_dimensions)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
        ])
    
    data_dir = 'PetImages'
    ds = datasets.ImageFolder(data_dir, transform, is_valid_file = is_valid)
    
    # https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split
    train_ds, test_ds = torch.utils.data.random_split(ds, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
    print(len(train_ds), len(test_ds))

    full_data_loaders = {
        'train': DataLoader(train_ds, batch_size=batch_size, 
                            worker_init_fn=seed_worker,
                            generator=g,
                            shuffle=True,
                            pin_memory=pin_memory,
                            num_workers=num_workers),
        'val': DataLoader(test_ds, batch_size=batch_size, shuffle=False,
                          pin_memory=pin_memory,
                          num_workers=num_workers)
    }
    class_names = ds.classes

    return full_data_loaders, class_names


# Experiments1 Base Class

In [6]:
class exp():
    def __init__(self, model_name: str, freeze_mode: int,
                full_data_loaders,
                 num_epochs: int, warmup_steps: int,  
                 batch_size: int = 32, 
                 opt: str = 'adam', lr: float = 0.001,  weight_decay=1e-5,
                 warmup_lr: float = 0.001, 
                 num_classes: int = 2, randseed: int = 11) -> None:
        
        self.seed = randseed
        torch.manual_seed(self.seed)
        random.seed(self.seed)
        np.random.seed(self.seed)
        
        self.full_data_loaders = full_data_loaders

        self.freeze_mode = freeze_mode
        self.model_name = model_name
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.opt_str = opt
        self.lr = lr
        self.warmup_lr = warmup_lr
        self.weight_decay = weight_decay
        self.num_classes = num_classes
        self.warmup_steps = warmup_steps

        self.model: nn.Module
        self.input_size: int
        self.params_to_update_str: List[str]
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        self.name = f'model_{model_name}-freeze_{freeze_mode}-optim_{opt}-{lr}-num_epochs_{num_epochs}-warmup_{self.warmup_steps}'
        self.init_env()

    def train_model(self, save_best = True): 
        # from https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html#enable-cudnn-auto-tuner
        torch.backends.cudnn.benchmark = True
        
        phases = ['train', 'val']
        # phases = ['train']
        
        since = time.time()

        val_acc_hist = []
        train_acc_hist = []
        val_loss_hist = []
        train_loss_hist = []

        best_model_wts = copy.deepcopy(self.model.state_dict())
        best_val_acc = 0.0
        best_val_acc_loss = 0.0
        best_epoch = 0

        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch, self.num_epochs - 1))
            print('-' * 10)

            if epoch == self.warmup_steps: 
                custom_set_parameter_requires_grad(self.model, self.model_name, self.freeze_mode)
                params_to_update = self.get_params_to_update()
                self.set_opt(params_to_update, False)

            for phase in phases:
                if phase == 'train':
                    self.model.train() 
                else:
                    self.model.eval()   

                running_loss = 0.0
                running_corrects = 0

                total_start = time.time()
                model_time = 0.0
                for inputs, labels in tqdm(self.full_data_loaders[phase]):
                    model_start = time.time()

                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)

                    self.optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = self.model(inputs)
                        loss = self.criterion(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            self.optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    model_time += time.time() - model_start
                    # print(model_time)

                print('total and model time and dataTime ', time.time() - total_start, model_time, (time.time() - total_start) -  model_time)

                epoch_loss = running_loss / len(self.full_data_loaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(self.full_data_loaders[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

                # deep copy the model
                if phase == 'val' and epoch_acc > best_val_acc:
                    best_val_acc = epoch_acc
                    best_model_wts = copy.deepcopy(self.model.state_dict())
                    best_epoch = epoch
                    best_val_acc_loss = epoch_loss
                if phase == 'val':
                    val_acc_hist.append(epoch_acc)
                    val_loss_hist.append(epoch_loss)
                elif phase == 'train':
                    train_acc_hist.append(epoch_acc)
                    train_loss_hist.append(epoch_loss)

            print()

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_val_acc), f'at epoch {best_epoch}')

        # load best model weights
        self.model.load_state_dict(best_model_wts)

        train_acc_hist = torch.asarray(train_acc_hist).cpu().detach().numpy()
        train_loss_hist = torch.asarray(train_loss_hist).cpu().detach().numpy()
        val_acc_hist = torch.asarray(val_acc_hist).cpu().detach().numpy()
        val_loss_hist = torch.asarray(val_loss_hist).cpu().detach().numpy()
        
        # SAVE
        mkdir(self.model_name)

        if save_best: 
            torch.save({
                'epoch': best_epoch,
                'model_name': self.model_name,
                'model': self.model,
                'optimizer': self.optimizer,
                'val_loss': best_val_acc_loss,
                'val_acc': best_val_acc, 
            }, f'{self.model_name}/{self.name}-model.pt')

        with open(f'{self.model_name}/{self.name}-stats.pickle', 'wb') as handle:
            pickle.dump({
                'model_name': self.model_name,
                'freeze_mode': self.freeze_mode,
                'num_epochs': self.num_epochs,
                'lr': self.lr,
                'optim': self.opt_str,
                'val_loss_hist': val_loss_hist,
                'val_acc_hist': val_acc_hist,
                'train_loss_hist': train_loss_hist, 
                'train_acc_hist': train_acc_hist,                 
            }, handle)

        plot_stats(train_loss_hist, val_loss_hist,
                   train_acc_hist, val_acc_hist,
                   f'{self.name}', 'images', f'{self.name}')

        return self.model

    def get_params_to_update(self):
        # get learnable parameters: 
        params_to_update = self.model.parameters()
        params_to_update_str = []
        print("Params to learn:")

        params_to_update = []
        for name,param in self.model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
                params_to_update_str.append(name)

        self.params_to_update = params_to_update
        self.params_to_update_str = params_to_update_str

        return params_to_update

    def set_opt(self, params_to_update, is_warmup: bool):
        lr_opt: float
        if is_warmup:
            lr_opt = self.warmup_lr
        else:
            lr_opt = self.lr

        if self.opt_str == 'sgd':
            self.optimizer = optim.SGD(params_to_update, lr=lr_opt, momentum=0.9, weight_decay=self.weight_decay)
        elif self.opt_str == 'adam':
            self.optimizer = optim.Adam(params_to_update, lr=lr_opt, weight_decay=self.weight_decay)
        else: 
            print('invalid optim')
            exit()

    def init_env(self):
        # model config
        self.model, self.input_size = self.init_model()

        self.model = self.model.to(self.device)
        
        params_to_update = self.get_params_to_update()
        # optims
        self.set_opt(params_to_update, True)

        # loss func
        self.criterion = nn.CrossEntropyLoss()

        # data loading 
        # self.full_data_loaders, class_names = get_data(self.batch_size, self.seed)

    def init_model(self, use_pretrained=True):
        # from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
        model_ft = None
        input_size = 0
        num_classes = self.num_classes
        
        # NO NON-LINEARITY in the last classifer, 
        # add two layers
        #  nn.Sequential(nn.Linear(model_resnet18.fc.in_features,512),
                                #   nn.ReLU(),
                                #   nn.Dropout(),
                                #   nn.Linear(512, num_classes))

        if self.model_name == 'oneLayer': 
            model_ft = oneLayer(224*224*3, 2)
            # model_ft = oneLayer(784, 10)
            
            input_size = 224

        if self.model_name == "resnet18":
            """ Resnet18
            """
            model_ft = models.resnet18(pretrained=use_pretrained)
            custom_set_parameter_requires_grad(model_ft, self.model_name, 100)
            num_ftrs = model_ft.fc.in_features
            model_ft.fc = nn.Sequential(
                nn.Linear(num_ftrs, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.8),
                nn.Linear(256, num_classes),
            )
            input_size = 224

        elif self.model_name == "vgg16":
            """ VGG16_bn
            """
            model_ft = models.vgg16_bn(pretrained=use_pretrained)
            custom_set_parameter_requires_grad(model_ft, self.model_name, 100)
            num_ftrs = model_ft.classifier[0].in_features
            model_ft.classifier = nn.Sequential(
                nn.Linear(num_ftrs, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.8),
                nn.Linear(256, num_classes),
            )
            input_size = 224
    
        elif self.model_name == "densenet121":
            """ Densenet121
            """
            model_ft = models.densenet121(pretrained=use_pretrained)
            custom_set_parameter_requires_grad(model_ft, self.model_name, 100)
            num_ftrs = model_ft.classifier.in_features
            model_ft.classifier = nn.Sequential(
                nn.Linear(num_ftrs, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.8),
                nn.Linear(256, num_classes),
            )
            input_size = 224
        else:
            print("Invalid model name, exiting...")
            # exit()
        
        print(model_ft)
        return model_ft, input_size

def custom_set_parameter_requires_grad(model: nn.Module, model_name: str, freeze_mode):
    # first freeze all layers: 
    for param in model.parameters():
        param.requires_grad = False

    if model_name == 'densenet121':
        unfreeze_layers = []
        if freeze_mode <= 100:
            for name, param in model.named_parameters():
                if 'classifier' in name: 
                    param.requires_grad = True
                    
        if freeze_mode <= 70:
            unfreeze_layers.append('norm5') # useless
            unfreeze_layers.append('denseblock4')
        if freeze_mode <= 50: 
            # ? 
            pass
        if freeze_mode <= 30: 
            unfreeze_layers.append('transition3')
            unfreeze_layers.append('denseblock3')
        if freeze_mode <= 10: 
            unfreeze_layers.append('transition2')
            unfreeze_layers.append('denseblock2')

        for name, param in model.named_parameters():
            unfreeze = False
            for layerName in unfreeze_layers: 
                if layerName in name:
                    unfreeze = True    
            if unfreeze: 
                param.requires_grad = True
            
    elif model_name == 'vgg16':
        unfreeze_layers_indices = []
        if freeze_mode <= 100:
            for name, param in model.named_parameters():
                if 'classifier' in name: 
                    param.requires_grad = True

        if freeze_mode <= 70:
            #  L5
            #  34-43
            unfreeze_layers_indices.extend(np.arange(34, 43+1).tolist())
        if freeze_mode <= 50: 
            # L4
            # 24-33
            unfreeze_layers_indices.extend(np.arange(24, 33+1).tolist())
        if freeze_mode <= 30: 
            # L3
            # 14-23
            unfreeze_layers_indices.extend(np.arange(14, 23+1).tolist())
        if freeze_mode <= 10: 
            # L2
            # 7-13
            unfreeze_layers_indices.extend(np.arange(7, 13+1).tolist())

        for layer_idx in unfreeze_layers_indices:
            for param in model.features[layer_idx].parameters():
                param.requires_grad = True    

    elif model_name == 'resnet18':
        unfreeze_layers = []
        if freeze_mode <= 100:
            unfreeze_layers.append('fc') 
        if freeze_mode <= 70:
            unfreeze_layers.append('avgpool') # useless
            unfreeze_layers.append('layer4')
        if freeze_mode <= 50: 
            unfreeze_layers.append('layer3')
        if freeze_mode <= 30: 
            unfreeze_layers.append('layer2')
        if freeze_mode <= 10: 
            unfreeze_layers.append('layer1')
        
        for name, param in model.named_parameters():
            unfreeze = False
            for layerName in unfreeze_layers: 
                if layerName in name:
                    unfreeze = True    
            if unfreeze: 
                param.requires_grad = True


# Experiments1


## ResNet


### resnet with warmups: 

In [ ]:
num_epochs = 10
lr_dict = {
    100: 0.001,
    70: 0.0005,
    50: 0.0001,
    30: 0.00005,
    10: 0.00001,
}
opt = 'sgd'
batch_size = 64
randseed = 11
warmup_steps = 1

full_loaders, _ = get_data(batch_size, randseed, pin_memory=True, num_workers=5)

19999 4999


In [ ]:
best_models = []
for model_name in ['resnet18']:
    for freeze_mode in [10, 30, 50, 70, 100]:
        if freeze_mode == 50 and model_name == 'densenet121': 
            continue
        print('-'*20)
        print('freeze_mode: ', freeze_mode)
        curr_exp = exp(model_name=model_name, freeze_mode=freeze_mode,
                        full_data_loaders = full_loaders, warmup_steps = warmup_steps,
                         num_epochs=num_epochs, batch_size=batch_size, 
                         opt=opt, lr = lr_dict[freeze_mode], warmup_lr = 0.001, randseed=randseed)
        best_models.append(curr_exp.train_model())

## VGG

### vgg with warmup

In [ ]:
num_epochs = 10
lr_dict = {
    100: 0.001,
    70: 0.0005,
    50: 0.0001,
    30: 0.00005,
    10: 0.00001,
}
opt = 'sgd'
batch_size = 64
randseed = 11
warmup_steps = 1

full_loaders, _ = get_data(batch_size, randseed, pin_memory=True, num_workers=5)

19999 4999


In [ ]:
best_models = []
for model_name in ['vgg16']:
    for freeze_mode in [100, 70, 50, 30, 10]: 
        if freeze_mode == 50 and model_name == 'densenet121': 
            continue
        print('-'*20)
        print('freeze_mode: ', freeze_mode)
        curr_exp = exp(model_name=model_name, freeze_mode=freeze_mode,
                        full_data_loaders = full_loaders, warmup_steps = warmup_steps,
                         num_epochs=num_epochs, batch_size=batch_size, 
                         opt=opt, lr = lr_dict[freeze_mode], warmup_lr = 0.001, randseed=randseed)
        best_models.append(curr_exp.train_model())

## DenseNet



### DenseNet w/warmup

In [ ]:
num_epochs = 10
lr_dict = {
    100: 0.001,
    70: 0.0005,
    50: 0.0001,
    30: 0.00005,
    10: 0.00001,
}
opt = 'sgd'
batch_size = 64
randseed = 11
warmup_steps = 1

full_loaders, _ = get_data(batch_size, randseed, pin_memory=True, num_workers=5)

In [ ]:
best_models = []
for model_name in ['densenet121']:
    for freeze_mode in [100, 70, 50, 30, 10]: 
        if freeze_mode == 50 and model_name == 'densenet121': 
            continue
        print('-'*20)
        print('freeze_mode: ', freeze_mode)
        curr_exp = exp(model_name=model_name, freeze_mode=freeze_mode,
                        full_data_loaders = full_loaders, warmup_steps = warmup_steps,
                         num_epochs=num_epochs, batch_size=batch_size, 
                         opt=opt, lr = lr_dict[freeze_mode], warmup_lr = 0.001, randseed=randseed)
        best_models.append(curr_exp.train_model())

# Experiments2 utils

In [8]:
def get_feats_dense(model, full_loaders, model_name):
    assert model_name == 'densenet121'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    features = {'train': [], 'val': []}
    all_labels = {'train': [], 'val': []}
    
    # hook
    def get_activation():
        def hook(model, input, output):
            features[phase].append(input[0].cpu().detach().numpy())
        return hook

    # best_model.layer4[1].bn2
    model.classifier[0].register_forward_hook(get_activation())

    debugged = False
    model = model.to(device)
    for phase in ['val', 'train']:
        model.eval()

        for inputs, labels in tqdm(full_loaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outs = model(inputs)

                if not debugged:
                    print(features[phase][0].shape)
                    debugged = True

            all_labels[phase].append(labels.cpu().detach().numpy())    

    features['train'] = np.concatenate(features['train'])
    features['val'] = np.concatenate(features['val'])
    all_labels['train'] = np.concatenate(all_labels['train'])
    all_labels['val'] = np.concatenate(all_labels['val'])
    
    return features, all_labels


def get_fe_model(model, model_name):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    if model_name == 'resnet18':
        return_nodes = {
            # "layer4.1.bn2": "features" ,
            "avgpool": "features",  
        }
        # m = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    elif model_name == 'vgg16':
        return_nodes = {
            "avgpool": "features",  
        }
    # elif model_name == 'densenet121':
    #     return_nodes = {
    #         "features": "features",  
    #     }
    
    fe_model = create_feature_extractor(model, return_nodes=return_nodes)
    return fe_model

def get_feats(fe_model, full_loaders, model_name):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    features = {'train': [], 'val': []}
    all_labels = {'train': [], 'val': []}
    
    debugged = False

    fe_model = fe_model.to(device)
    for phase in ['val', 'train']:
        fe_model.eval()

        for inputs, labels in tqdm(full_loaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                feats = fe_model(inputs)['features']

                if not debugged:
                    print(feats.shape)
                    debugged = True

                features[phase].append(feats.cpu().detach().numpy())
                all_labels[phase].append(labels.cpu().detach().numpy())    


    features['train'] = np.concatenate(features['train'])
    features['val'] = np.concatenate(features['val'])
    all_labels['train'] = np.concatenate(all_labels['train'])
    all_labels['val'] = np.concatenate(all_labels['val'])
    

    if model_name == 'resnet18':
        features['train'] = features['train'].reshape((features['train'].shape[0], features['train'].shape[1]))
        features['val'] = features['val'].reshape((features['val'].shape[0], features['val'].shape[1]))
        
    elif model_name == 'vgg16':
        print(features['train'].shape)
        features['train'] = features['train'].reshape((features['train'].shape[0], features['train'].shape[1]*features['train'].shape[2]*features['train'].shape[3]))
        features['val'] = features['val'].reshape((features['val'].shape[0], features['val'].shape[1]*features['val'].shape[2]*features['val'].shape[3]))
        
    return features, all_labels

def test_model(model, fdl):
    model.eval()   
    running_corrects = 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for inputs, labels in tqdm(fdl['val']):
        model_start = time.time()

        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / len(fdl['val'].dataset)
    print(epoch_acc)
    print(running_corrects.double(), len(fdl['val'].dataset))
    return epoch_acc

# Experiments2 Base class: 

In [10]:
class RandomForrestCLF:
    def __init__(self, model_name: str, freeze_mode: int, full_loaders):
        self.model_name = model_name
        self.freeze_mode = freeze_mode
        self.best_saved: Dict
        self.base_dir = 'randomForrest'
        self.full_loaders = full_loaders
        mkdir(self.base_dir)

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    def load_model(self):
        num_epochs = 10
        lr_dict = {
            100: 0.001,
            70: 0.0005,
            50: 0.0001,
            30: 5e-05,
            10: 1e-05,
        }
        opt = 'sgd'
        batch_size = 64
        randseed = 11
        warmup_steps = 1

        self.best_saved = torch.load(f'{self.model_name}/model_{self.model_name}-freeze_{self.freeze_mode}-optim_{opt}-{lr_dict[self.freeze_mode]}-num_epochs_{num_epochs}-warmup_{warmup_steps}-model.pt')
        """
        torch.save({
            'epoch': best_epoch,
            'model_name': self.model_name,
            'model': self.model,
            'optimizer': self.optimizer,
            'val_loss': best_val_acc_loss,
            'val_acc': best_val_acc, 
        }, f'{self.model_name}/{self.name}-model.pt')
        """

    def save_feats(self):
        
        if self.model_name == 'resnet18':
            self.feature_extractor = get_fe_model(self.best_saved['model'], self.model_name)
            self.features, self.labels = get_feats(self.feature_extractor, self.full_loaders, self.model_name)
        elif self.model_name == 'vgg16': 
            self.feature_extractor = get_fe_model(self.best_saved['model'], self.model_name)
            self.features, self.labels = get_feats(self.feature_extractor, self.full_loaders, self.model_name)
        elif self.model_name == 'densenet121': 
            self.features, self.labels = get_feats_dense(self.best_saved['model'], self.full_loaders, self.model_name)

        mkdir(f'{self.base_dir}/{self.model_name}')
        np.savez(f'{self.base_dir}/{self.model_name}/features_{self.model_name}-{self.freeze_mode}.npz', 
                 features_train=self.features['train'],
                 features_val  =self.features['val'],
                 labels_train  =self.labels['train'],
                 labels_val    =self.labels['val'])
    
    def train_clf(self, n_estimators, max_depth):
        if not os.path.isfile(f'{self.base_dir}/{self.model_name}/features_{self.model_name}-{self.freeze_mode}.npz'):
            print('features from model', self.model_name, 'freeze ', self.freeze_mode, ' not saved, extracting features')
            self.load_model()
            self.save_feats()

        
        results = np.load(f'{self.base_dir}/{self.model_name}/features_{self.model_name}-{self.freeze_mode}.npz')
        self.features = {
            'train': results['features_train'],
            'val':   results['features_val']
        }
        self.labels = {
            'train': results['labels_train'],
            'val':   results['labels_val']
        }

        self.clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=11)
        self.clf.fit(self.features['train'], self.labels['train'])
        acc = self.clf.score(self.features['val'], self.labels['val'])

        print(f'using model {self.model_name} with freeze_mode {self.freeze_mode} as feature extractor...')
        print(f'RF params: n_estimators {n_estimators}, max_depth {max_depth}')
        print('acc: ', acc)

# train RandomForrest

In [12]:
full_loaders, _ = get_data(64, 11, pin_memory=True, num_workers=5)

19999 4999


### Resnet
best of resnet18 -> freeze 70

In [14]:
rfclf = RandomForrestCLF('resnet18', 70, full_loaders)
for n_estimators in [5, 10, 20, 50]:
    for max_depth in [10, 50, 100, 200]:
        rfclf.train_clf(n_estimators, max_depth)

using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 10
acc:  0.9783956791358271
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 50
acc:  0.9779955991198239
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 100
acc:  0.9779955991198239
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 200
acc:  0.9779955991198239
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 10, max_depth 10
acc:  0.9813962792558512
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 10, max_depth 50
acc:  0.9827965593118624
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estimators 10, max_depth 100
acc:  0.9827965593118624
using model resnet18 with freeze_mode 70 as feature extractor...
RF params: n_estima

###Densenet
best of densenet121 -> freeze 30

In [16]:
rfclf = RandomForrestCLF('densenet121', 70, full_loaders)
for n_estimators in [5, 10, 20, 50]:
    for max_depth in [10, 50, 100, 200]:
        rfclf.train_clf(n_estimators, max_depth)

using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 10
acc:  0.9751950390078016
using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 50
acc:  0.9749949989997999
using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 100
acc:  0.9749949989997999
using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 5, max_depth 200
acc:  0.9749949989997999
using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 10, max_depth 10
acc:  0.9817963592718544
using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 10, max_depth 50
acc:  0.9809961992398479
using model densenet121 with freeze_mode 70 as feature extractor...
RF params: n_estimators 10, max_depth 100
acc:  0.9809961992398479
using model densenet121 with freeze_mode 70 as feature extracto

###VGG
best of vgg16 -> freeze 30

In [17]:
rfclf = RandomForrestCLF('vgg16', 30, full_loaders)
for n_estimators in [5, 10, 20, 50]:
    for max_depth in [10, 50, 100, 200]:
        rfclf.train_clf(n_estimators, max_depth)

using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 5, max_depth 10
acc:  0.9585917183436687
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 5, max_depth 50
acc:  0.9587917583516703
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 5, max_depth 100
acc:  0.957991598319664
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 5, max_depth 200
acc:  0.957991598319664
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 10, max_depth 10
acc:  0.9707941588317663
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 10, max_depth 50
acc:  0.9729945989197839
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 10, max_depth 100
acc:  0.9737947589517904
using model vgg16 with freeze_mode 30 as feature extractor...
RF params: n_estimators 10, max_depth 200
acc

# Old data loading code

In [ ]:
# class customDS(Dataset):
#     def __init__(self, data, labels, transforms=None):
#         self.data = data
#         self.labels = labels
#         self.transforms = transforms

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         if self.transforms is None:
#             return self.data[idx], self.labels[idx]
#         else:
#             return self.transforms(self.data[idx]), self.labels[idx]

# def get_dataloaders(batch_size: int = 8, randseed: int = 11, pret_model_data: str = 'ImageNet'):
#     """
#     if not customDataSet is saved on disk:
#         create ImageFolder Dataset, 
#         Read and transform Data,
#         save the data into list,
#         create customDataSet from the lists,
#         torch save the customDataSets,
#     torch load customDataSets from disk,
#     create data loaders
#     return data loaders 
#     """
#     if not (os.path.isfile('dataset/train_ds.pt') \
#             and os.path.isfile('dataset/test_ds.pt')):

#         full_loaders_1, _ = get_data(256, randseed=randseed, \
#                                      pin_memory=True, pret_model_data=pret_model_data)

#         data_train = []
#         y_train = []
#         data_test = []
#         y_test = []
#         for phase in ['train', 'val']:
#             idx = 0
#             for inputs, labels in tqdm(full_loaders_1[phase]):
#                 idx += 1
#                 if phase == 'train':
#                     data_train.append(inputs)
#                     y_train.append(labels)
#                 else:
#                     data_test.append(inputs)
#                     y_test.append(labels)
#                 # if idx == 5:
#                 #     break 
         
#         train_ds_temp = customDS(torch.cat(data_train), torch.cat(y_train))
#         test_ds_temp  = customDS(torch.cat(data_test), torch.cat(y_test))

#         mkdir('dataset')
#         torch.save(train_ds_temp, 'dataset/train_ds.pt')
#         torch.save(test_ds_temp, 'dataset/test_ds.pt')

#     train_ds = torch.load('dataset/train_ds.pt')
#     test_ds  = torch.load('dataset/test_ds.pt')

#     def seed_worker(worker_id):
#         # worker_seed = torch.initial_seed() % 2 ** 32
#         np.random.seed(randseed)
#         random.seed(randseed)

#     g = torch.Generator()
#     g.manual_seed(randseed)

#     full_data_loaders = {
#         'train': DataLoader(train_ds, batch_size=batch_size, 
#                             worker_init_fn=seed_worker,
#                             generator=g,
#                             shuffle=True,
#                             # pin_memory=pin_memory
#                             ),
#         'val': DataLoader(test_ds, batch_size=batch_size, shuffle=False,
#                           # pin_memory=pin_memory
#                           )
#     }

#     return full_data_loaders